<a href="https://colab.research.google.com/github/Apekshya-Shakya/TECH405/blob/main/RNN_Gold_Price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:
import pandas as pd
import numpy as np

In [45]:
df = pd.read_csv('/content/drive/MyDrive/College_assignments/TECH405 Artificial Neural Network and Deep Learning/Gold Price (2013-2023).csv')


In [46]:
df.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,12/30/2022,"1,826.20","1,821.80","1,832.40","1,819.80",107.50K,0.01%
1,12/29/2022,"1,826.00","1,812.30","1,827.30","1,811.20",105.99K,0.56%
2,12/28/2022,"1,815.80","1,822.40","1,822.80","1,804.20",118.08K,-0.40%
3,12/27/2022,"1,823.10","1,808.20","1,841.90","1,808.00",159.62K,0.74%
4,12/26/2022,"1,809.70","1,805.80","1,811.95","1,805.55",NaN,0.30%


In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2583 entries, 0 to 2582
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Date      2583 non-null   object
 1   Price     2583 non-null   object
 2   Open      2583 non-null   object
 3   High      2583 non-null   object
 4   Low       2583 non-null   object
 5   Vol.      2578 non-null   object
 6   Change %  2583 non-null   object
dtypes: object(7)
memory usage: 141.4+ KB


In [48]:
df['Date'] = pd.to_datetime(df['Date'])
df.sort_values(by='Date', ascending=True, inplace=True)

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2583 entries, 2582 to 0
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      2583 non-null   datetime64[ns]
 1   Price     2583 non-null   object        
 2   Open      2583 non-null   object        
 3   High      2583 non-null   object        
 4   Low       2583 non-null   object        
 5   Vol.      2578 non-null   object        
 6   Change %  2583 non-null   object        
dtypes: datetime64[ns](1), object(6)
memory usage: 161.4+ KB


In [50]:
df.drop(['Vol.', 'Change %'], axis=1, inplace=True)

In [60]:
df_clean = df.replace({',': ''}, regex=True)


In [62]:
df[['Price', "Open", "High", "Low"]] = df_clean[['Price', "Open", "High", "Low"]].astype('float64')

In [63]:
df.isnull().sum()

,0
Date,0
Price,0
Open,0
High,0
Low,0


In [64]:
df['Price'].describe()

,Price
count,2583.000000
mean,1440.326442
std,257.338282
min,1049.600000
25%,1244.200000
50%,1320.500000
75%,1713.950000
max,2058.400000


In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2583 entries, 2582 to 0
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    2583 non-null   datetime64[ns]
 1   Price   2583 non-null   float64       
 2   Open    2583 non-null   float64       
 3   High    2583 non-null   float64       
 4   Low     2583 non-null   float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 121.1 KB


In [72]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.layers import SimpleRNN

In [67]:
prices = df[['Date', 'Price']].set_index('Date')

In [68]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_prices = scaler.fit_transform(prices)

In [69]:
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

In [70]:
sequence_length = 60
X, y = create_sequences(scaled_prices, sequence_length)

In [71]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

In [75]:
model = Sequential([
    SimpleRNN(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    SimpleRNN(50, return_sequences=False),
    Dense(25),
    Dense(1)
])

In [76]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [77]:
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1, verbose=1)

Epoch 1/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - loss: 0.0442 - val_loss: 0.0377
Epoch 2/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 0.0012 - val_loss: 0.0171
Epoch 3/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 39ms/step - loss: 0.0010 - val_loss: 0.0111
Epoch 4/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 6.5921e-04 - val_loss: 0.0092
Epoch 5/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 5.3170e-04 - val_loss: 0.0045
Epoch 6/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 5.5039e-04 - val_loss: 0.0061
Epoch 7/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 5.8410e-04 - val_loss: 0.0036
Epoch 8/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 4.1734e-04 - val_loss: 0.0028
Epoch 9/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 3.9687e-04 - val_loss: 0.0029
Epoch 10/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 3.9268e-04 - val_loss: 0.0034
Epoch 11/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 3.4733e-04 - val_loss: 0.0022
Epoch 12/20
57/57 ━━━━━━━━━━━━━

In [79]:
loss = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 8.3819e-04
Test Loss: 0.0009017384727485478


In [80]:
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
